In [50]:
import itertools, json
from IPython.display import clear_output

def find_item_by_id(item_id, data):
    search = list(filter(lambda x: x['id'] == item_id, data))
    if(len(search) == 1):
        return search[0]
    elif(len(search) > 1):
        raise Exception("Not find.")
    
def get_id_users(data):
    return list(map(lambda x: x['id'], data))
    
def get_movies(user_id, data):
    user = find_item_by_id(user_id, data)
    return user['filmes']

def non_watch_movies(user_movies, neighborhood, data):
    all_nb_movies = list(map(lambda x: get_movies(x[0], data), neighborhood)) # all movies seen by the neighborhood
    all_nb_movies = list(itertools.chain.from_iterable(all_nb_movies)) # [[]] -> []
    all_nb_movies = list(set(map(lambda x: x['id'], all_nb_movies))) # IDs of all neighborhood movies
    u_movies = list(map(lambda x: x['id'], user_movies)) # IDs of user movies
    
    return list(filter(lambda x: x not in u_movies, all_nb_movies)) # movies not seen by the user in his neighborhood

def just_famous_movies(neighborhood, movies, n_movies, data): # (neighborhood) nearest neighborhood (e.g. 40 nearest users by similarity)
    nb_ids = list(map(lambda x: x[0], neighborhood)) # id of (id, similarity)
    nb = list(filter(lambda x: x['id'] in nb_ids, data)) # neighbors by id
    filmes_nb = list(map(lambda x: x['filmes'], nb)) # rating of neighbors [[]]
    filmes_nb = list(itertools.chain.from_iterable(filmes_nb)) # [[]] -> []
    
    if len(filmes_nb) == 0:
        return {'famous_movies': []}
    
    filmes_nb = list(map(lambda x: x['id'], filmes_nb)) # IDs of filmes    
    filmes_nb = list(map(lambda x: (x, filmes_nb.count(x)), movies)) # (movie, frequency)
    filmes_nb.sort(key=lambda x: x[1], reverse=True) # sort by highest frequency
    famous_movies = filmes_nb[:n_movies] # n popular movies
    famous_movies = list(map(lambda x: x[0], famous_movies)) # only IDs, frequency is not necessary, just for ranking
    
#     if len(famous_movies) > 0: # hard to happen, but...
        # search neighbors who watched at least one popular movie
    nb_watched_famous_movies = list(filter(lambda x: any(movie in famous_movies for movie in list(map(lambda y: y['id'], x['filmes'])) ), nb))
        
        # return in 'nb' the users obj to ease later, but it can be just IDs
    return {'famous_movies': famous_movies, 'nb': nb_watched_famous_movies}
#     else:
#         print("O valor de n = ", n, " é muito grande. Não existe um filme que ", n, " vizinhos assistiram.")
#         return {}
    
def find_similarity(neighborhood, user_id):
    search = list(filter(lambda x: x[0] == user_id, neighborhood))
    if(len(search) == 1):        
        return search[0][1] # return only similarity value
    else:
        raise Exception('not find')

all_predict = []
unpredictable = []
dataset_test = json.load(open('./test.json'))
def predict_user_movie(user, n, min_movies, data):
#     n = 2 # neighborhood
#     min_movies = 1 # min value of filmes
    
    neighborhood = user['user_similarity']
    neighborhood.sort(key=lambda x: x[1], reverse=True) # sort by similarity
    neighborhood = neighborhood[:n] # the real neighborhood
    
    # movies watched by neighborhood but not by the user
#     non_watched_movies = non_watch_movies(user['filmes'], neighborhood, data)

    # just the movies with high frequency within neighborhood
#     famous_movies = just_famous_movies(neighborhood, non_watched_movies, min_movies, data)
    
#     if famous_movies['famous_movies'] == []:
#         unpredictable.append({'user_id': user['id']})
#     else:
#         nb = famous_movies['nb'] # users
#         famous_movies = famous_movies['famous_movies'] # movies IDs

#         nb_ids = list(map(lambda x: x['id'], nb)) # just IDs
#         neighborhood = list(filter(lambda x: x[0] in nb_ids, neighborhood)) # filter by IDs -> (user_id, similarity)

    pa_is = [] # final prediction (movie_id, prediction_value)
    
    nb = list(map(lambda x: find_item_by_id(x[0],data), neighborhood))
    main_user = find_item_by_id(user['id'], dataset_test)
    famous_movies = list(map(lambda x: x['id'], main_user['filmes']))
    
    for movie_id in famous_movies:
        sum_1 = 0
        sum_2 = 0
        for i in nb:
            try:
                sim = find_similarity(neighborhood, i['id']) # search in list [(user_id, similarity)]
                mv = find_item_by_id(movie_id, i['filmes']) # retrieve movie in user movies list
                sum_1 = sum_1 + ((mv['avaliacao'] - i['media']) * sim)
                sum_2 = sum_2 + sim
            except:
#                 sum_1 = sum_1 + ((0 - i['media']) * sim)
                sum_2 = sum_2 + sim
        if sum_2 > 0:
            pa_i = user['media'] + (sum_1/sum_2)
        else:
            pa_i = user['media']
        pa_is.append({'filme':movie_id, 'predicao': pa_i})

    all_predict.append({'id': user['id'], 'predicoes': pa_is})

    
dataset = json.load(open('./usuarios.json'))
users = get_id_users(dataset)
tam = len(users)
for id_user in users:
    clear_output()
    print("Restam:", tam-1)
    tam = tam-1
    user = find_item_by_id(id_user, dataset)
    predict_user_movie(user, 20, 5, dataset)

all_predict[:5]

# with open('predictable.json', 'w') as outfile:
#     json.dump(all_predict, outfile)
    
# with open('unpredictable.json', 'w') as outfile:
#     json.dump(unpredictable, outfile)

Restam: 0


[{'id': 1488844,
  'predicoes': [{'filme': '17', 'predicao': 2.883454734651405}]},
 {'id': 30878, 'predicoes': [{'filme': '18', 'predicao': 2.8650933464891755}]},
 {'id': 823519, 'predicoes': [{'filme': '8', 'predicao': 2.466666666666667}]},
 {'id': 1181550,
  'predicoes': [{'filme': '16', 'predicao': 3.5203571428571427}]},
 {'id': 1227322,
  'predicoes': [{'filme': '8', 'predicao': 2.3877638144027324},
   {'filme': '17', 'predicao': 2.3960884762133463}]}]

# Separação dos dados disponíveis em treinamento e testes

In [1]:
from collections import Counter
import json, math
from random import randrange

dataset = json.load(open('./combined_data_1_1M.json'))

def reservoir_sampling(items, k):
    """ 
    Reservoir sampling algorithm for large sample space or unknow end list
    See <http://en.wikipedia.org/wiki/Reservoir_sampling> for detail>
    Type: ([a] * Int) -> [a]
    Prev constrain: k is positive and items at least of k items
    Post constrain: the length of return array is k
    Credits: https://gist.github.com/m00nlight/bfe54d1b2db362755a3a
    """
    sample = items[0:k]
    
    for i in range(k, len(items)):
        j = randrange(1, i + 1)
        if j < k:
            sample[j] = items[i]

    return sample

filtered = list(filter(lambda x: len(x['filmes']) >= 3, dataset))
# filtered = list(filter(lambda x: len(x['filmes']) >= 24, dataset)) # for tests

trainset = []
testset = []
for item in filtered:
    train = reservoir_sampling(item['filmes'], math.floor(len(item['filmes'])*.75))
    test = list(filter(lambda x: x not in train, item['filmes']))
    trainset.append({'id': item['id'], 'media': item['media'], 'user_similarity': [], 'filmes': train})
    testset.append({'id': item['id'], 'media': item['media'], 'user_similarity': [], 'filmes': test})

with open('train.json', 'w') as trainfile:
    json.dump(trainset, trainfile)
with open('test.json', 'w') as testfile:
    json.dump(testset, testfile)

In [10]:
import json
with open('train.json') as f:
    usuarios = json.load(f)
print(len(usuarios))

80000


In [ ]:
import copy, json, math
    
print("start")
lambda_differ = 0.5

def inter(f1, f2):
    ids = set(map(lambda x: x['id'], f2)).intersection(set(map(lambda x: x['id'], f1)))
    return list(ids)

def calculo(user, num):
    global usuarios
    global file
    global lambda_differ
    
    na = 0
    user_similarity = []
    for i in usuarios:
        if user['id'] != i['id']:
            somatorio_1 = 0
            somatorio_2 = 0
            somatorio_3 = 0
            sumdiffer = 0
            m = 0
            ids = inter(user['filmes'], i['filmes'])
            if len(ids) > 0:
                f1 = list(filter(lambda x: x['id'] in ids, user['filmes']))
                f2 = list(filter(lambda x: x['id'] in ids, i['filmes']))
                for x,y in enumerate(f1):
                    s1 = (y["avaliacao"] - user["media"])
                    s2 = (f2[x]["avaliacao"] - i["media"])
                    somatorio_1 += s1*s2
                    somatorio_2 += math.pow(s1, 2)
                    somatorio_3 += math.pow(s2, 2)
                    m = m + 1
                    sumdiffer += math.pow(y["avaliacao"] - f2[x]["avaliacao"], 2)
                raiz = math.sqrt( somatorio_2 * somatorio_3)
                if (m):
                    sumdiffer = math.sqrt( sumdiffer/m)
                else:
                    sumdiffer = 0
                if (raiz): #verifica divizao por zero
                    similaridade = somatorio_1 / raiz
                else:
                    similaridade = 0
                if(m < 50):
                    similaridade = similaridade / (50-m)
                similaridade = similaridade *  math.pow(lambda_differ, sumdiffer)
                if(similaridade > 0):
                    user_similarity.append((i["id"], similaridade))
#                     i["user_similarity"].append((user["id"], similaridade))
            else:
                na += 1
            
    user_similarity.sort(key=lambda x: x[1], reverse=True)
    temp = copy.deepcopy(user)
    temp['user_similarity'] = user_similarity[:50]
    with open('usuariosteste.json', 'a') as outfile:
        outfile.write(json.dumps(temp)+",\n")
        outfile.close()
    print("num:", num, "\tcompleto:", (((num+1)*100)/80000),"%","\tSuficiente?",na<=79950)

with open('usuariosteste.json', 'a') as outfile:
    outfile.write("[\n")
    outfile.close()
    
tam = len(usuarios)
for i, user in enumerate(usuarios):
    calculo(user, i)
    
with open('usuariosteste.json', 'a') as outfile:
    outfile.write("]")
    outfile.close()

start
num: 0 	completo: 0.00125 % 	Suficiente? True
num: 1 	completo: 0.0025 % 	Suficiente? True
num: 2 	completo: 0.00375 % 	Suficiente? True
num: 3 	completo: 0.005 % 	Suficiente? True
num: 4 	completo: 0.00625 % 	Suficiente? True
num: 5 	completo: 0.0075 % 	Suficiente? True
num: 6 	completo: 0.00875 % 	Suficiente? True
num: 7 	completo: 0.01 % 	Suficiente? True
num: 8 	completo: 0.01125 % 	Suficiente? True
num: 9 	completo: 0.0125 % 	Suficiente? True
num: 10 	completo: 0.01375 % 	Suficiente? True
num: 11 	completo: 0.015 % 	Suficiente? True
num: 12 	completo: 0.01625 % 	Suficiente? True
num: 13 	completo: 0.0175 % 	Suficiente? True
num: 14 	completo: 0.01875 % 	Suficiente? True
num: 15 	completo: 0.02 % 	Suficiente? True
num: 16 	completo: 0.02125 % 	Suficiente? True
num: 17 	completo: 0.0225 % 	Suficiente? True
num: 18 	completo: 0.02375 % 	Suficiente? True
num: 19 	completo: 0.025 % 	Suficiente? True
num: 20 	completo: 0.02625 % 	Suficiente? True
num: 21 	completo: 0.0275 % 	Sufic

num: 177 	completo: 0.2225 % 	Suficiente? True
num: 178 	completo: 0.22375 % 	Suficiente? True
num: 179 	completo: 0.225 % 	Suficiente? True
num: 180 	completo: 0.22625 % 	Suficiente? True
num: 181 	completo: 0.2275 % 	Suficiente? True
num: 182 	completo: 0.22875 % 	Suficiente? True
num: 183 	completo: 0.23 % 	Suficiente? True
num: 184 	completo: 0.23125 % 	Suficiente? True
num: 185 	completo: 0.2325 % 	Suficiente? True
num: 186 	completo: 0.23375 % 	Suficiente? True
num: 187 	completo: 0.235 % 	Suficiente? True
num: 188 	completo: 0.23625 % 	Suficiente? True
num: 189 	completo: 0.2375 % 	Suficiente? True
num: 190 	completo: 0.23875 % 	Suficiente? True
num: 191 	completo: 0.24 % 	Suficiente? True
num: 192 	completo: 0.24125 % 	Suficiente? True
num: 193 	completo: 0.2425 % 	Suficiente? True
num: 194 	completo: 0.24375 % 	Suficiente? True
num: 195 	completo: 0.245 % 	Suficiente? True
num: 196 	completo: 0.24625 % 	Suficiente? True
num: 197 	completo: 0.2475 % 	Suficiente? True
num: 198 	

num: 351 	completo: 0.44 % 	Suficiente? True
num: 352 	completo: 0.44125 % 	Suficiente? True
num: 353 	completo: 0.4425 % 	Suficiente? True
num: 354 	completo: 0.44375 % 	Suficiente? True
num: 355 	completo: 0.445 % 	Suficiente? True
num: 356 	completo: 0.44625 % 	Suficiente? True
num: 357 	completo: 0.4475 % 	Suficiente? True
num: 358 	completo: 0.44875 % 	Suficiente? True
num: 359 	completo: 0.45 % 	Suficiente? True
num: 360 	completo: 0.45125 % 	Suficiente? True
num: 361 	completo: 0.4525 % 	Suficiente? True
num: 362 	completo: 0.45375 % 	Suficiente? True
num: 363 	completo: 0.455 % 	Suficiente? True
num: 364 	completo: 0.45625 % 	Suficiente? True
num: 365 	completo: 0.4575 % 	Suficiente? True
num: 366 	completo: 0.45875 % 	Suficiente? True
num: 367 	completo: 0.46 % 	Suficiente? True
num: 368 	completo: 0.46125 % 	Suficiente? True
num: 369 	completo: 0.4625 % 	Suficiente? True
num: 370 	completo: 0.46375 % 	Suficiente? True
num: 371 	completo: 0.465 % 	Suficiente? True
num: 372 	co

num: 525 	completo: 0.6575 % 	Suficiente? True
num: 526 	completo: 0.65875 % 	Suficiente? True
num: 527 	completo: 0.66 % 	Suficiente? True
num: 528 	completo: 0.66125 % 	Suficiente? True
num: 529 	completo: 0.6625 % 	Suficiente? True
num: 530 	completo: 0.66375 % 	Suficiente? True
num: 531 	completo: 0.665 % 	Suficiente? True
num: 532 	completo: 0.66625 % 	Suficiente? True
num: 533 	completo: 0.6675 % 	Suficiente? True
num: 534 	completo: 0.66875 % 	Suficiente? True
num: 535 	completo: 0.67 % 	Suficiente? True
num: 536 	completo: 0.67125 % 	Suficiente? True
num: 537 	completo: 0.6725 % 	Suficiente? True
num: 538 	completo: 0.67375 % 	Suficiente? True
num: 539 	completo: 0.675 % 	Suficiente? True
num: 540 	completo: 0.67625 % 	Suficiente? True
num: 541 	completo: 0.6775 % 	Suficiente? True
num: 542 	completo: 0.67875 % 	Suficiente? True
num: 543 	completo: 0.68 % 	Suficiente? True
num: 544 	completo: 0.68125 % 	Suficiente? True
num: 545 	completo: 0.6825 % 	Suficiente? True
num: 546 	c

num: 699 	completo: 0.875 % 	Suficiente? True
num: 700 	completo: 0.87625 % 	Suficiente? True
num: 701 	completo: 0.8775 % 	Suficiente? True
num: 702 	completo: 0.87875 % 	Suficiente? True
num: 703 	completo: 0.88 % 	Suficiente? True
num: 704 	completo: 0.88125 % 	Suficiente? True
num: 705 	completo: 0.8825 % 	Suficiente? True
num: 706 	completo: 0.88375 % 	Suficiente? True
num: 707 	completo: 0.885 % 	Suficiente? True
num: 708 	completo: 0.88625 % 	Suficiente? True
num: 709 	completo: 0.8875 % 	Suficiente? True
num: 710 	completo: 0.88875 % 	Suficiente? True
num: 711 	completo: 0.89 % 	Suficiente? True
num: 712 	completo: 0.89125 % 	Suficiente? True
num: 713 	completo: 0.8925 % 	Suficiente? True
num: 714 	completo: 0.89375 % 	Suficiente? True
num: 715 	completo: 0.895 % 	Suficiente? True
num: 716 	completo: 0.89625 % 	Suficiente? True
num: 717 	completo: 0.8975 % 	Suficiente? True
num: 718 	completo: 0.89875 % 	Suficiente? True
num: 719 	completo: 0.9 % 	Suficiente? True
num: 720 	com

...

num: 52699 	completo: 65.875 % 	Suficiente? True
num: 52700 	completo: 65.87625 % 	Suficiente? True
num: 52701 	completo: 65.8775 % 	Suficiente? True
num: 52702 	completo: 65.87875 % 	Suficiente? True
num: 52703 	completo: 65.88 % 	Suficiente? True
num: 52704 	completo: 65.88125 % 	Suficiente? True
num: 52705 	completo: 65.8825 % 	Suficiente? True
num: 52706 	completo: 65.88375 % 	Suficiente? True
num: 52707 	completo: 65.885 % 	Suficiente? True
num: 52708 	completo: 65.88625 % 	Suficiente? True
num: 52709 	completo: 65.8875 % 	Suficiente? True
num: 52710 	completo: 65.88875 % 	Suficiente? True
num: 52711 	completo: 65.89 % 	Suficiente? True
num: 52712 	completo: 65.89125 % 	Suficiente? True
num: 52713 	completo: 65.8925 % 	Suficiente? True
num: 52714 	completo: 65.89375 % 	Suficiente? True
num: 52715 	completo: 65.895 % 	Suficiente? True
num: 52716 	completo: 65.89625 % 	Suficiente? True
num: 52717 	completo: 65.8975 % 	Suficiente? True
num: 52718 	completo: 65.89875 % 	Suficiente? Tr

num: 52863 	completo: 66.08 % 	Suficiente? True
num: 52864 	completo: 66.08125 % 	Suficiente? True
num: 52865 	completo: 66.0825 % 	Suficiente? True
num: 52866 	completo: 66.08375 % 	Suficiente? True
num: 52867 	completo: 66.085 % 	Suficiente? True
num: 52868 	completo: 66.08625 % 	Suficiente? True
num: 52869 	completo: 66.0875 % 	Suficiente? True
num: 52870 	completo: 66.08875 % 	Suficiente? True
num: 52871 	completo: 66.09 % 	Suficiente? True
num: 52872 	completo: 66.09125 % 	Suficiente? True
num: 52873 	completo: 66.0925 % 	Suficiente? True
num: 52874 	completo: 66.09375 % 	Suficiente? True
num: 52875 	completo: 66.095 % 	Suficiente? True
num: 52876 	completo: 66.09625 % 	Suficiente? True
num: 52877 	completo: 66.0975 % 	Suficiente? True
num: 52878 	completo: 66.09875 % 	Suficiente? True
num: 52879 	completo: 66.1 % 	Suficiente? True
num: 52880 	completo: 66.10125 % 	Suficiente? True
num: 52881 	completo: 66.1025 % 	Suficiente? True
num: 52882 	completo: 66.10375 % 	Suficiente? True

num: 53027 	completo: 66.285 % 	Suficiente? True
num: 53028 	completo: 66.28625 % 	Suficiente? True
num: 53029 	completo: 66.2875 % 	Suficiente? True
num: 53030 	completo: 66.28875 % 	Suficiente? True
num: 53031 	completo: 66.29 % 	Suficiente? True
num: 53032 	completo: 66.29125 % 	Suficiente? True
num: 53033 	completo: 66.2925 % 	Suficiente? True
num: 53034 	completo: 66.29375 % 	Suficiente? True
num: 53035 	completo: 66.295 % 	Suficiente? True
num: 53036 	completo: 66.29625 % 	Suficiente? True
num: 53037 	completo: 66.2975 % 	Suficiente? True
num: 53038 	completo: 66.29875 % 	Suficiente? True
num: 53039 	completo: 66.3 % 	Suficiente? True
num: 53040 	completo: 66.30125 % 	Suficiente? True
num: 53041 	completo: 66.3025 % 	Suficiente? True
num: 53042 	completo: 66.30375 % 	Suficiente? True
num: 53043 	completo: 66.305 % 	Suficiente? True
num: 53044 	completo: 66.30625 % 	Suficiente? True
num: 53045 	completo: 66.3075 % 	Suficiente? True
num: 53046 	completo: 66.30875 % 	Suficiente? Tru

num: 53191 	completo: 66.49 % 	Suficiente? True
num: 53192 	completo: 66.49125 % 	Suficiente? True
num: 53193 	completo: 66.4925 % 	Suficiente? True
num: 53194 	completo: 66.49375 % 	Suficiente? True
num: 53195 	completo: 66.495 % 	Suficiente? True
num: 53196 	completo: 66.49625 % 	Suficiente? True
num: 53197 	completo: 66.4975 % 	Suficiente? True
num: 53198 	completo: 66.49875 % 	Suficiente? True
num: 53199 	completo: 66.5 % 	Suficiente? True
num: 53200 	completo: 66.50125 % 	Suficiente? True
num: 53201 	completo: 66.5025 % 	Suficiente? True
num: 53202 	completo: 66.50375 % 	Suficiente? True
num: 53203 	completo: 66.505 % 	Suficiente? True
num: 53204 	completo: 66.50625 % 	Suficiente? True
num: 53205 	completo: 66.5075 % 	Suficiente? True
num: 53206 	completo: 66.50875 % 	Suficiente? True
num: 53207 	completo: 66.51 % 	Suficiente? True
num: 53208 	completo: 66.51125 % 	Suficiente? True
num: 53209 	completo: 66.5125 % 	Suficiente? True
num: 53210 	completo: 66.51375 % 	Suficiente? True

num: 53355 	completo: 66.695 % 	Suficiente? True
num: 53356 	completo: 66.69625 % 	Suficiente? True
num: 53357 	completo: 66.6975 % 	Suficiente? True
num: 53358 	completo: 66.69875 % 	Suficiente? True
num: 53359 	completo: 66.7 % 	Suficiente? True
num: 53360 	completo: 66.70125 % 	Suficiente? True
num: 53361 	completo: 66.7025 % 	Suficiente? True
num: 53362 	completo: 66.70375 % 	Suficiente? True
num: 53363 	completo: 66.705 % 	Suficiente? True
num: 53364 	completo: 66.70625 % 	Suficiente? True
num: 53365 	completo: 66.7075 % 	Suficiente? True
num: 53366 	completo: 66.70875 % 	Suficiente? True
num: 53367 	completo: 66.71 % 	Suficiente? True
num: 53368 	completo: 66.71125 % 	Suficiente? True
num: 53369 	completo: 66.7125 % 	Suficiente? True
num: 53370 	completo: 66.71375 % 	Suficiente? True
num: 53371 	completo: 66.715 % 	Suficiente? True
num: 53372 	completo: 66.71625 % 	Suficiente? True
num: 53373 	completo: 66.7175 % 	Suficiente? True
num: 53374 	completo: 66.71875 % 	Suficiente? Tru